In [1]:
import numpy as np

In [2]:
def read_files(folder_path='/home/grad/ccomp/21/nuneslima/Datasets/Netherlands/data'):
    train_label=np.load(folder_path+"/train/train_labels.npy")
    train_seismic=np.load(folder_path+"/train/train_seismic.npy") 
    train_seismic =((train_seismic+1)/2)*255
    train_seismic=train_seismic.astype(np.uint8)

    test1_label=np.load(folder_path+"/test_once/test1_labels.npy")
    test1_seismic=np.load(folder_path+"/test_once/test1_seismic.npy")    
    test1_seismic =((test1_seismic+1)/2)*255
    test1_seismic=test1_seismic.astype(np.uint8)

    test2_label=np.load(folder_path+"/test_once/test2_labels.npy")
    test2_label=np.transpose(test2_label,(1,0,2))
    test2_seismic=np.load(folder_path+"/test_once/test2_seismic.npy")
    test2_seismic=np.transpose(test2_seismic,(1,0,2))
    test2_seismic =((test2_seismic+1)/2)*255
    test2_seismic=test2_seismic.astype(np.uint8)


    return train_label, train_seismic, test1_label, test1_seismic, test2_label, test2_seismic

In [3]:
def divide_into_patches(images, patch_size, stride):
    num_images, height, width = images.shape
    
    patches_per_dim_h = (height - patch_size) // stride + 1
    patches_per_dim_w = (width - patch_size) // stride + 1
    
    num_patches_per_image = patches_per_dim_h * patches_per_dim_w
    
    patches = np.zeros((num_images * num_patches_per_image, patch_size, patch_size), dtype=images.dtype)
    
    idx = 0
    for image in images:
        for h in range(0, height - patch_size + 1, stride):
            for w in range(0, width - patch_size + 1, stride):
                patch = image[h:h+patch_size, w:w+patch_size]
                patches[idx] = patch
                idx += 1
                
    return patches

In [4]:
def penobscot_data(patch,stride):
    train_label, train_seismic, test1_label, test1_seismic, test2_label, test2_seismic = read_files()
    patch_mask_train = divide_into_patches(train_label,patch,stride)
    patch_train = divide_into_patches(train_seismic,patch,stride)
    patch_mask_test = divide_into_patches(test1_label,patch,stride)
    patch_test = divide_into_patches(test1_seismic,patch,stride)
    patch_mask_val = divide_into_patches(test2_label,patch,stride)
    patch_val = divide_into_patches(test2_seismic,patch,stride)

    return patch_train, patch_mask_train, patch_test, patch_mask_test, patch_val, patch_mask_val


In [5]:
def majority_class(array):
    n, a, b = array.shape
    majority_classes = []

    for i in range(n):
        sample = array[i]
        flattened_sample = sample.flatten()
        unique_classes, counts = np.unique(flattened_sample, return_counts=True)
        majority_class_index = np.argmax(counts)
        majority_class = unique_classes[majority_class_index]
        majority_classes.append(majority_class)

    return np.array(majority_classes)

In [6]:
patch_train, patch_mask_train, patch_test, patch_mask_test, patch_val, patch_mask_val = penobscot_data(50,30)

In [7]:
train_label, train_seismic, test1_label, test1_seismic, test2_label, test2_seismic = read_files()
print(train_label.shape,train_seismic.shape)
print(test1_label.shape,test1_seismic.shape)
print(test2_label.shape,test2_seismic.shape)

(401, 701, 255) (401, 701, 255)
(200, 701, 255) (200, 701, 255)
(200, 601, 255) (200, 601, 255)


In [8]:
import matplotlib.pyplot as plt

In [9]:
print(patch_train.shape,majority_class(patch_mask_train).shape)
print(patch_test.shape,majority_class(patch_mask_test).shape)
print(patch_val.shape,majority_class(patch_mask_val).shape)

(61754, 50, 50) (61754,)
(30800, 50, 50) (30800,)
(26600, 50, 50) (26600,)
